# Projet 2 Participez à un concours sur la Smart City

## Jupyter Notebook

Nous utilisons ce programme, qui est un outil informatique de logiciel libre, qui est basé sur IPython. Il vous permettra de lire les commentaires écrit au format markdown, et d'interpréter les cellules de code, ici en Python. Mais cette technologie marche avec d'autres langages, Scala par exemple.

## Pandas

Nous utiliserons le langage Python, conformément à la consigne, et nous importerons la bibliothèque de manipulation de données Pandas. Ensuite nous importerons Numpy pour travailler avec des matrices.

In [ ]:
import datashader as ds
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import pandasql as ps
import seaborn as sns

import colorcet
import itertools
import math
import openpyxl
import random
import scipy

from collections import OrderedDict, deque
from IPython.display import HTML as html_print
from sklearn import preprocessing


# 1) Présentation générale du jeu de données

## Data requirements

La ville de Paris veut optimiser ses tournées, il s'agit d'un problème d'algorithmie courant : ***Problème du voyageur de commerce***. Aujourd'hui ce calcul peut s'effectuer avec plusieurs millions de ville sans que cela ne prenne trop de temps de calcul. La ville de Paris met à notre disposition les positions de chaque emplacement et leur nom, de sorte que si nous considérons que le réseau de Paris est assez dense pour que nous puissions négliger les sinuosités de la route, nous pouvons calculer la distance à vol d'oiseau, entre les emplacements. Et en prenant l'emplacement le plus proche de l'emplacement ou nous sommes, nous pouvons espérer trouver ainsi le plus court chemin pour parvenir à notre objectif de tournée.

## Data collection

Nous récupérons les données depuis une seule source, un fichier "comma separator value" ou csv, [à cette adresse](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+2+Participez+%C3%A0+un+concours+sur+la+Smart+City/p2-arbres-fr.csv). Nous plaçons ensuite le fichier dans le répertoire de travail pour l'importer dans la partie suivante. Ce jeu de données comporte $17$ colonnes. $200137$ lignes et $646164$ valeurs manquantes. La colonne numéro est inutilisable, et celle des id ne démarre pas à *0*. Plus d'information seront détaillées dans les paragraphes suivants.

## Data processing

Nous obtenons les données dans le bon format, mais avec quelques erreurs, nous verrons cela plus tard, pour l'instant nous importons nos données en indiquant le séparateur, et la colonne servant à indexer la table de nos données. Nous remplaçons la colonne d'indexation par un index à l'itération parfaite. Nous éliminons les colonnes id et numero qui ne sont pas intéressante pour nos observations.

In [ ]:
data = pd.read_csv('data.csv', sep=';')
data.drop('id', axis=1, inplace=True)
data.drop('numero', axis=1, inplace=True)

## Data cleaning

Nous devons contrôler les doublons, les valeurs nulles, les données incomplètes, les erreurs et les données manquantes. Nous réglons les types, pour obtenir des valeurs à mettre en base de données, au besoin.
Voilà comment se présentent les données sorties du fichier.

In [ ]:
data.info()

#### Les valeurs nulles, les types.

Nous remplaçons les valeurs nulles de la colonne "remarquable" par la valeur 0 (qui code pour False), et nous pouvons remplacer le type par un booléen après avoir échanger les valeurs flottantes par des booleénnes.

In [ ]:
data['remarquable'].fillna(value=0, inplace=True)
data['remarquable'] = data['remarquable'].map({0.:False, 1.:True})
data['remarquable'] = data['remarquable'].convert_dtypes()

Pour les colonnes "arrondissement"…, qui n'ont pas de valeurs nulles, nous forçons la conversion

In [ ]:
colonnes = ['arrondissement', 'type_emplacement', 'lieu', 'id_emplacement']
for col in colonnes:
    data[col] = data[col].convert_dtypes(convert_string=True)

Pour les colonnes "stade_developpement"…, comme ce sont des strings, nous remplaçons les valeurs nulles par deux quote.

In [ ]:
colonnes = ['stade_developpement', 'espece', 'variete', 'genre', 'libelle_francais',\
            'complement_addresse', 'domanialite']
for col in colonnes:
    data[col].fillna(value="", inplace=True)
    data[col] = data[col].convert_dtypes(convert_string=True)

In [ ]:
data.info()

#### Construire une table d'information personnalisée

Il est possible de se passer de la méthode info pour construire un algorithme plus puissant afin d'observer le plus d'information sur les tables :

In [ ]:
def informations(data):
    # Header
    print('{:^3}{:<1}{:^20}{:<1}{:^9}{:>1}{:^6}{:<1}{:^15}{:<1}{:^7}{:<1}{:^7}{:<1}'.format('#' , '|' , 'Column', '|', 'Dtype', '|', 'Unique', '|', 'Count Non-Null','|','Mean','|','Std','|'))
    print('{:^3}{:<1}{:^20}{:<1}{:^9}{:>1}{:^6}{:<1}{:^15}{:<1}{:^7}{:<1}{:^7}{:<1}'.format('---' , '|' , '------', '|', '-----', '|', '------', '|', '--------------','|','----','|','---','|'))
    # dtypes information
    dtypes_uniques = set() # Collection of unique elements
    dtypes_listes = [] # list of complete data columns
    memory_usage = 0 # in MB
    for it, col in enumerate(data.columns): # Feed the set and the list
        dtypes_listes.append(str(data[col].dtype))
        if str(data[col].dtype) not in dtypes_uniques:
            dtypes_uniques.add(str(data[col].dtype))
        # Table body
        print('{:^3}{:<1}{:<20}{:<1}{:^9}{:>1}{:<6}{:<1}{:<15}{:<1}{:<7.5}{:<1}{:<7.5}{:<1}'.format(\
                                                str(it), '|' , col, '|',\
                                                str(data[col].dtype), '|', str(len(data[col].unique())),\
                                                '|',str(data[col].count()) + ' Non-null','|',\
                                                str(data[col].mean())\
                                                    if (data[col].dtype in ['int64','float64']) else '','|',\
                                                str(data[col].std())\
                                                    if (data[col].dtype in ['int64','float64']) else '','|'))
        
        # Collect informatives on disk usage by observation onto the data column
        memory_usage += int(data[col].memory_usage(index=True, deep=False))
    # Blend of set and list to print the information line as usual
    dtypes_string = ''
    for x in dtypes_uniques:
        dtypes_string += '{}({}), '.format(x, dtypes_listes.count(x))
    print('\ndtypes: {}'.format(dtypes_string))
    # Digit format to write mem usage in comprehensive format
    print('\nmemory usage: {:.4} MB\n'.format(memory_usage / (1024*1024)))

In [ ]:
informations(data)

# 2) Démarche méthodologique d'analyse de données

## Analyse univarée

Notre analyse va nous servir à détecter les erreurs dans la base de données, qui sont autant anomalies ("outliers") ou de données aberrantes qui peuvent nous conduire à de fausses conclusions sur les éléments de notre analyse. Nous analyserons les colonnes circonférence en centimètre, et hauteur en mètre. Les ordres de grandeurs sont indiqués dans les lignes min/max du tableau ci-dessous. À noter que nous commençons d'abord par décrire les données sans filtres :

In [ ]:
data[['circonference_cm','hauteur_m']].describe()

### Intervalle interquartile

$Q_i = Q_3 - Q_1$ nous donne l'intervalle interquartile. L'intervalle minimum de confiance est $Q_i \times 1.5$, c'est la limite intérieur. De plus l'intervalle maximum de confiance est $Q_i \times 3$. Nous réglons nos bornes selon cette formule : pour la circonférence le maximum est $255$, alors que pour la hauteur le maximum est $21$.

Ci-dessus, la ligne max nous permet de raisonner sur la présence d'aberrations et dans les prochains paragraphes, en ajustant les valeurs d'exclusions en multipliant par 3,5 , nous allons faire une analyse univariée pour la circonférence et la hauteur des arbres de la ville de Paris. Nous avons pris la circonférence entre $]0, 255] centimètres$, la hauteur entre $]0, 21] mètres$, puis les remarquables avec une valeur Vrai (dans un cas spécialement adapté ou la corrélation est évaluée entre la hauteur et la circonférence).

### La circonférence

D'après le graphique et les résultats qui suivent, la circonférence de la population est une distribution dont le maximum est de $400 cm$ pour une moyenne de 91 cm et un écart type de $57,59 cm$ de part et d'autre de la moyenne. L'allure et la répartition de la distribution de la population est une asymétrie vers la droite, et le plus fréquemment, la circonférence est de 20 cm. L'échantillon considéré est de $174026$ arbres.

In [ ]:
a = data.loc[(data["circonference_cm"] <= 255) & (data["circonference_cm"] > 0)]
sns.displot(data=a, x="circonference_cm", kde=True)

In [ ]:
a["circonference_cm"].describe()

In [ ]:
a["circonference_cm"].mode()

In [ ]:
scipy.stats.skew(a["circonference_cm"], axis=0, bias=True, nan_policy='propagate')

In [ ]:
scipy.stats.kurtosis(a["circonference_cm"], axis=0, bias=True, nan_policy='propagate')

### La hauteur

La distribution est irrégulière, mais contient $160122$ arbres après filtrage des données aberrantes. La hauteur maximale est $29 mètres$ et la moyenne est de $10,32 mètres$, tout comme le mode et la médiane. Par contre l'asymétrie est positive, donc vers la droite mais sa valeur est inférieur à $1$, comme l'est aussi son kurtosis. Visuellement, il serait difficile de dire qu'ici s'applique le théorème de limite centrale car nous n'avons pas a première vue une Gaussienne.


In [ ]:
b = data.loc[(data["hauteur_m"] < 21) & (data["hauteur_m"] > 0)]
sns.displot(data=b, x="hauteur_m", kde=True)

In [ ]:
b["hauteur_m"].describe()

In [ ]:
b["hauteur_m"].mode()

In [ ]:
scipy.stats.skew(b["hauteur_m"], axis=0, bias=True, nan_policy='propagate')

In [ ]:
scipy.stats.kurtosis(b["hauteur_m"], axis=0, bias=True, nan_policy='propagate')

### Régression linéaire

En considérant l'expression de la circonférence d'après la hauteur, nous pouvons constater une corrélation positive entre le données quantitative, autant pour les arbres remarquables, que les premiers.

In [ ]:
sns.regplot( x="hauteur_m", y="circonference_cm", data=data.loc[((data["hauteur_m"] < 21) & (data["hauteur_m"] > 0)) \
                          & ((data["circonference_cm"] < 255) & (data["circonference_cm"] > 0)) & (data["remarquable"] == True)]);

Si nous ajoutons les variables indépendantes circonférence, hauteur pour les arbres remarquables, nous pourrions sans doute appliquer le théorème de limite centrale sur nos données ainsi filtrés tant les courbes ont une forme dont la régularité s'approche plus d'une gaussienne. Peut-être nous restera-t-il à croiser les données avec d'autres pour affiner la courbe en forme de cloche.

In [ ]:
c = data.loc[((data["hauteur_m"] < 30) & (data["hauteur_m"] > 0)) \
                          & ((data["circonference_cm"] < 400) & (data["circonference_cm"] > 0)) & data["remarquable"] == True]

In [ ]:
c[["hauteur_m","circonference_cm"]].describe()

Pour la circonférence des arbres remarquables, le plus fréquemment, elle vaut $200 cm$ et $15 m$ pour la hauteur.

In [ ]:
sns.displot(data=c, x="circonference_cm", kde=True)

In [ ]:
c["circonference_cm"].mode()

In [ ]:
scipy.stats.skew(c["circonference_cm"], axis=0, bias=True, nan_policy='propagate')

In [ ]:
scipy.stats.kurtosis(c["circonference_cm"], axis=0, bias=True, nan_policy='propagate')

In [ ]:
sns.displot(data=c, x="hauteur_m", kde=True)

La description des valeurs du graphique ci-dessus est donnée ci-dessous. Elle nous permet de comparer **les ordres de grandeur** entre les colonnes.

In [ ]:
c["hauteur_m"].mode()

In [ ]:
scipy.stats.skew(c["hauteur_m"], axis=0, bias=True, nan_policy='propagate')

In [ ]:
scipy.stats.kurtosis(c["hauteur_m"], axis=0, bias=True, nan_policy='propagate')


## Analyse Quantitative

Nous utilisons un filtre sur la base pour éliminer les arbres dont nous ne sommes pas sur de la hauteur ou de la circonférence, puis avec une requête SQL, nous déterminons le nombre d'individu, dans chaque élément quantitatif à l'intérieur de chaque quartier, excepté pour la domanialité qui est une données de catégorie. Nous ajoutons en plus des moyennes ainsi que des pourcentages du nombre d'arbres. Nous commençons notre analyse quantitative sur $159568$ arbres.

In [ ]:
data_filtered = data.loc[((data["hauteur_m"] < 30) & (data["hauteur_m"] > 0)) \
                          & ((data["circonference_cm"] < 400) & (data["circonference_cm"] > 0))].copy()

De plus nous remarquons qu'une ligne ne comporte pas de domanialité, mais qu'il semble d'après son lieu, que ce soit un jardin.

In [ ]:
data_filtered.loc[data_filtered['domanialite'] == ""]

In [ ]:
mask = data_filtered['domanialite'] == ''
data_filtered.loc[mask,'domanialite'] = 'Jardin'

In [ ]:
data_filtered.loc[data_filtered['domanialite'] == ""]

De même pour la variété, pour 63000 arbres :

In [ ]:
mask = data_filtered['variete'] == ''
data_filtered.loc[mask,'variete'] = 'n. sp.'
mask = data_filtered['espece'] == ''
data_filtered.loc[mask,'espece'] = 'n. sp.'

De plus pour la légende des stades de développement, nous rajoutons la P, pour particulier, car il représente une masse d'arbre qui sorte des statistiques pour la moitié d'entre eux. Cette catégorie passe de 60000 arbres à 30000 arbres après filtrage des hauteurs et des circonférences. Cette catégorie semble regroupé des arbres de catégories JA et J.

In [ ]:
data.stade_developpement = data.stade_developpement.map({'':'P', 'A':'A','JA':'JA','M':'M','J':'J'})
data_filtered.stade_developpement = data_filtered.stade_developpement.map({'':'P', 'A':'A','JA':'JA','M':'M','J':'J'})

In [ ]:
data.stade_developpement.unique()

Comme la colonne de stade de développement est une variable qualitative ordonnée, nous créons une colonne de sa valeur en chiffre pour un usage quantitatif de la variable car l'âge est quantitatif.

In [ ]:
data_filtered['maturite'] = data_filtered.stade_developpement.map({'P':2, 'A':4,'JA':3,'M':5,'J':1})

In [ ]:
count_before_filtering = data['type_emplacement'].loc[data.stade_developpement == ''].count()
count_after_filtering = data_filtered['type_emplacement'].loc[data_filtered.stade_developpement == 'P'].count()
print('Avant : ', count_before_filtering, 'arbres, Après filtrage : ', count_after_filtering, 'arbres')

In [ ]:
q1 = """SELECT  arrondissement,
                ROUND(Count(type_emplacement)* 100. / (Select Count(type_emplacement) From data), 2) as "pourcent",
                count(DISTINCT(domanialite)) as domani,
                count(DISTINCT(lieu)) as lieu,
                count(type_emplacement) as arbres,
                count(DISTINCT(id_emplacement)) as id_empl,
                count(DISTINCT(libelle_francais)) as libel_fr,
                count(DISTINCT(genre)) as genre,
                count(DISTINCT(espece)) as espece,
                count(DISTINCT(variete)) as variete,
                ROUND(AVG(circonference_cm),2) as circon,
                ROUND(AVG(hauteur_m),2) as haut,
                SUM(remarquable) as remarq
                FROM data_filtered GROUP BY data_filtered.arrondissement"""

In [ ]:
quant_arr = ps.sqldf(q1, locals())

In [ ]:
quant_arr

In [ ]:
sns.set_theme(style="whitegrid")

df = quant_arr[['haut','circon', 'arbres', 'arrondissement']]

f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.despine(f, left=True, bottom=True)
#clarity_ranking = ['M','A','JA','P','J']
sns.scatterplot(x="circon", y="haut",
                hue="arrondissement", #hue_order=clarity_ranking,
                size="arbres",
                data=df, ci="sd",ax=ax)

Nous faisons la même chose pour chaque domanialité :

In [ ]:
q2 = """SELECT  domanialite,
                ROUND(Count(type_emplacement)* 100. / (Select Count(type_emplacement) From data),2) as "pourcent",
                count(DISTINCT(arrondissement)) as arrond,
                count(DISTINCT(stade_developpement)) as devel,
                count(DISTINCT(lieu)) as lieu,
                count(type_emplacement) as arbres,
                count(DISTINCT(id_emplacement)) as id_empl,
                count(DISTINCT(libelle_francais)) as libel_fr,
                count(DISTINCT(genre)) as genre,
                count(DISTINCT(espece)) as espece,
                count(DISTINCT(variete)) as variete,
                ROUND(AVG(circonference_cm),2) as circon,
                ROUND(AVG(hauteur_m),2) as haut,
                SUM(remarquable) as remarq
                FROM data_filtered GROUP BY data_filtered.domanialite"""

In [ ]:
quant_dom = ps.sqldf(q2, locals())

In [ ]:
quant_dom

Nous observons que le stade de développement peut indiquer une relation avec les produits à utiliser, donc nous utilisons le stade de développement comme pivot.

In [ ]:
q3 = """SELECT  stade_developpement,
                ROUND(Count(type_emplacement)* 100. / (Select Count(type_emplacement) From data),2) as "pourcent",
                count(DISTINCT(arrondissement)) as arrond,
                count(DISTINCT(domanialite)) as domani,
                count(DISTINCT(lieu)) as lieu,
                count(type_emplacement) as arbres,
                count(DISTINCT(id_emplacement)) as id_empl,
                count(DISTINCT(libelle_francais)) as libel_fr,
                count(DISTINCT(genre)) as genre,
                count(DISTINCT(espece)) as espece,
                count(DISTINCT(variete)) as variete,
                ROUND(AVG(circonference_cm), 2) as circon,
                ROUND(AVG(hauteur_m), 2) as haut,
                SUM(remarquable) as remarq
                FROM data_filtered GROUP BY data_filtered.stade_developpement"""

In [ ]:
quant_dev = ps.sqldf(q3, locals())

In [ ]:
quant_dev

Ici nous exprimons les quantités par stade de développement, et par domanialité

In [ ]:
q4 = """SELECT  stade_developpement as devel,
                domanialite as domani,
                ROUND(Count(type_emplacement)* 100. / (Select Count(type_emplacement) From data), 2) as "pourcent",
                count(DISTINCT(arrondissement)) as arrond,
                count(DISTINCT(lieu)) as lieu,
                count(type_emplacement) as arbres,
                count(DISTINCT(id_emplacement)) as id_empl,
                count(DISTINCT(libelle_francais)) as libel_fr,
                count(DISTINCT(genre)) as genre,
                count(DISTINCT(espece)) as espece,
                count(DISTINCT(variete)) as variete,
                ROUND(AVG(circonference_cm),2) as circon,
                ROUND(AVG(hauteur_m),2) as haut,
                SUM(remarquable) as remarq
                FROM data_filtered GROUP BY data_filtered.stade_developpement, domanialite
                ORDER BY devel, domani, haut"""

In [ ]:
quant_dev_dom = ps.sqldf(q4, locals())

In [ ]:
quant_dev_dom

In [ ]:
sns.set_theme(style="whitegrid")

df = quant_dev_dom[['haut','circon', 'arbres', 'devel']]

f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.despine(f, left=True, bottom=True)
clarity_ranking = ['M','A','JA','P','J']
sns.scatterplot(x="haut", y="circon",
                hue="devel", hue_order=clarity_ranking,
                size="arbres",
                data=df, ci="sd",ax=ax)

In [ ]:
q5 = """SELECT  genre,
                ROUND(Count(type_emplacement)* 100. / (Select Count(type_emplacement) From data),2) as "pourcent",
                count(DISTINCT(arrondissement)) as arrond,
                count(DISTINCT(domanialite)) as domani,
                count(DISTINCT(lieu)) as lieu,
                count(type_emplacement) as arbres,
                count(DISTINCT(id_emplacement)) as id_empl,
                count(DISTINCT(libelle_francais)) as libel_fr,
                count(DISTINCT(variete)) as variete,
                count(DISTINCT(espece)) as espece,
                ROUND(AVG(maturite), 2) as cat_age,
                ROUND(AVG(circonference_cm), 2) as circon,
                ROUND(AVG(hauteur_m), 2) as haut,
                SUM(remarquable) as remarq
                FROM data_filtered GROUP BY data_filtered.genre"""

In [ ]:
quant_gen = ps.sqldf(q5, locals())

In [ ]:
quant_gen

In [ ]:
x = quant_gen[['haut','circon','cat_age','arbres']].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)


In [ ]:
df = df.rename(columns={0:'haut',1:'circon',2:'cat_age',3:'arbres'})

In [ ]:
quant_gen = df.merge(quant_gen['genre'], on=df.index)

In [ ]:
quant_gen.drop('key_0',axis=1, inplace=True)

In [ ]:
sns.set_theme(style="whitegrid")

# Load the dataset
crashes = quant_gen[['genre','arbres','cat_age','haut', 'circon']]#sns.load_dataset("car_crashes")

# Make the PairGrid
g = sns.PairGrid(crashes.sort_values("circon", ascending=True),
                 x_vars=crashes.columns[1:], y_vars=["genre"],
                 height=20, aspect=.25)

# Draw a dot plot using the stripplot function
g.map(sns.stripplot, size=10, orient="h", jitter=False,
      palette="flare_r", linewidth=1, edgecolor="w")

# Use the same x axis limits on all columns and add better labels
g.set(xlim=(0, 1), xlabel='Norme', ylabel="")

# Use semantically meaningful titles for the columns
titles = ['nb arbres','age','haut', 'circonférence']

for ax, title in zip(g.axes.flat, titles):

    # Set a different title for each axes
    ax.set(title=title)

    # Make the grid horizontal instead of vertical
    ax.xaxis.grid(False)
    ax.yaxis.grid(True)

sns.despine(left=True, bottom=True)
g.savefig("genre_circonf.png")


In [ ]:
sns.set_theme(style="whitegrid")

# Load the dataset
crashes = quant_gen[['genre','arbres','cat_age','haut', 'circon']]#sns.load_dataset("car_crashes")

# Make the PairGrid
g = sns.PairGrid(crashes.sort_values("haut", ascending=True),
                 x_vars=crashes.columns[1:], y_vars=["genre"],
                 height=20, aspect=.25)

# Draw a dot plot using the stripplot function
g.map(sns.stripplot, size=10, orient="h", jitter=False,
      palette="flare_r", linewidth=1, edgecolor="w")

# Use the same x axis limits on all columns and add better labels
g.set(xlim=(0, 1), xlabel='Norme', ylabel="")

# Use semantically meaningful titles for the columns
titles = ['nb arbres','age','hauteur', 'circonférence']

for ax, title in zip(g.axes.flat, titles):

    # Set a different title for each axes
    ax.set(title=title)

    # Make the grid horizontal instead of vertical
    ax.xaxis.grid(False)
    ax.yaxis.grid(True)

sns.despine(left=True, bottom=True)
g.savefig("genre_hauteur.png")


In [ ]:
sns.set_theme(style="whitegrid")

# Load the dataset
crashes = quant_gen[['genre','arbres','cat_age','haut', 'circon']]#sns.load_dataset("car_crashes")

# Make the PairGrid
g = sns.PairGrid(crashes.sort_values("cat_age", ascending=True),
                 x_vars=crashes.columns[1:], y_vars=["genre"],
                 height=20, aspect=.25)

# Draw a dot plot using the stripplot function
g.map(sns.stripplot, size=10, orient="h", jitter=False,
      palette="flare_r", linewidth=1, edgecolor="w")

# Use the same x axis limits on all columns and add better labels
g.set(xlim=(0, 1), xlabel='Norme', ylabel="")

# Use semantically meaningful titles for the columns
titles = ['nb arbres','age','haut', 'circonférence']

for ax, title in zip(g.axes.flat, titles):

    # Set a different title for each axes
    ax.set(title=title)

    # Make the grid horizontal instead of vertical
    ax.xaxis.grid(False)
    ax.yaxis.grid(True)

sns.despine(left=True, bottom=True)
g.savefig("genre_age.png")


In [ ]:
sns.set_theme(style="whitegrid")

# Load the dataset
crashes = quant_gen[['genre','arbres','cat_age','haut', 'circon']]#sns.load_dataset("car_crashes")

# Make the PairGrid
g = sns.PairGrid(crashes.sort_values("arbres", ascending=True),
                 x_vars=crashes.columns[1:], y_vars=["genre"],
                 height=20, aspect=.25)

# Draw a dot plot using the stripplot function
g.map(sns.stripplot, size=10, orient="h", jitter=False,
      palette="flare_r", linewidth=1, edgecolor="w")

# Use the same x axis limits on all columns and add better labels
g.set(xlim=(0, 1), xlabel='Norme', ylabel="")

# Use semantically meaningful titles for the columns
titles = ['nb arbres','age','haut', 'circonférence']

for ax, title in zip(g.axes.flat, titles):

    # Set a different title for each axes
    ax.set(title=title)

    # Make the grid horizontal instead of vertical
    ax.xaxis.grid(False)
    ax.yaxis.grid(True)

sns.despine(left=True, bottom=True)
g.savefig("genre_nb_arbres.png")


# 3) Synthèse de mon analyse

In [ ]:
sns.set_theme(style="whitegrid")

df = quant_dev_dom[['haut','circon', 'arbres', 'devel']]

f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.despine(f, left=True, bottom=True)
clarity_ranking = ['M','A','JA','P','J']
sns.scatterplot(x="haut", y="circon",
                hue="devel", hue_order=clarity_ranking,
                size="arbres",
                data=df, ci="sd",ax=ax)

In [ ]:
sns.set_theme(style="whitegrid")

df = quant_arr[['haut','circon', 'arbres', 'arrondissement']]

f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.despine(f, left=True, bottom=True)
#clarity_ranking = ['M','A','JA','P','J']
sns.scatterplot(x="circon", y="haut",
                hue="arrondissement", #hue_order=clarity_ranking,
                size="arbres",
                data=df, ci="sd",ax=ax)

In [ ]:
sns.set_theme(style="whitegrid")

df = quant_dom[['haut','circon', 'arbres', 'domanialite']]

f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.despine(f, left=True, bottom=True)
#clarity_ranking = ['M','A','JA','P','J']
sns.scatterplot(x="circon", y="haut",
                hue="domanialite", #hue_order=clarity_ranking,
                size="arbres",
                data=df, ci="sd",ax=ax)

## Carte des positions

In [ ]:
def cstr(s, color='green'):
    return "<text style=color:{}>{}</text>".format(color,s)
html_print("Some " + cstr('green'))

In [ ]:
couleurs_list = ['#4ffb28',
         '#5fa390',
         '#ff1515',
         '#694231',#'#5b3d2a',
         '#000000'
         ]

In [ ]:
couleurs_dict = dict()
for it, val in enumerate(data['stade_developpement'].unique()):
    couleurs_dict.update({val: couleurs_list[it]})
couleurs_dict

In [ ]:
data['couleurs'] = data['stade_developpement'].map(couleurs_dict)

In [ ]:
html_print(cstr('J: #ff1515', '#ff1515') + ",\n"
 +cstr("P: #4ffb28", "#4ffb28")+',\n'
 +cstr('JA: #000000', '#000000')+',\n'
 +cstr('A: #5fa390', '#5fa390')+',\n'
 +cstr('M: #694231', '#694231')+'}\n' )

In [ ]:
cvs = ds.Canvas(plot_width=1000, plot_height=1000)
agg = cvs.points(data, 'geo_point_2d_b', 'geo_point_2d_a')
img = ds.tf.shade(agg, cmap=list(data['couleurs']), how='eq_hist')
img

# Partie Algorithmie, Optimisation des trajets

Nous avons réalisé une analyse exploratoire de nos données, à présent, nous allons tâcher de répondre à la problèmatique du meilleur trajet, qui est un problème similaire au problème du voyageur de commerce pour lequel nous souhaitons utiliser l'algorithme de [Christofides](https://fr.wikipedia.org/wiki/Algorithme_de_Christofides). Pour cela nous utiliserons un GPU en local, pour calculer les distances entre tous les lieux, et nous lancerons l'algorithme de recherche du plus court chemin entre tous les lieux soit $6921$ éléments distincts.

Nous devons suivre le schéma suivant pour construire l'algorithme de Christofides :



![Algorithme de Christofides](christofides.png)

## Calcul du meilleur trajet

### Calcul des distances entre chaque lieu

Nous relevons 200137 points d'intérêts sur la carte. Si nous descendons dans la structure hiérarchisé de nos données, nous comprenons qu'il s'agit geo_point_2d -> id_emplacement -> lieu + complement d'adresse -> lieu -> arrondissement. C'est à dire qu'un lieu regroupe des id_emplacement, i.e. des arbres.

Comme il s'agit de trouver une tournée véhiculée, entre $6921$ lieux uniques, contenant eux-mêmes un certain nombre d'arbres (chacun), et que ces lieux sont reliés entre eux par une distance pouvant se calculer d'après la latitude et la longitude, nous utiliserons une représentation en graphe (arrête et sommet) pour trouver le meilleurs chemins, en commençant par le calcul de la matrice des distances, puis du chemin de poids minimum.

1. Nous calculerons la quantité d'arbres d'un lieu par la fonction d'aggrégeage "aggfunc" ci-dessous. Nous disposerons le résultat dans une colonne portant le nom de aire, car c'est une donnée quantitative que nous souhaitons représenter sous la forme de cercle avec plus ou moins de surface.

In [ ]:
df = data[['arrondissement', 'lieu', 'geo_point_2d_a', 'geo_point_2d_b']].copy()

In [ ]:
lieu_aire = pd.DataFrame(df.pivot_table(index=['lieu'], aggfunc='size'), columns=['aire'])

In [ ]:
x = lieu_aire.reset_index()

**Nous éliminons ainsi les lieux doublons**, notons que les bulles de circonférences sont en rapport avec le nombre de ligne (et donc d'arbre) disparaissant dans cette opération.

In [ ]:
df = df.drop_duplicates(subset='lieu', ignore_index=True)

In [ ]:
set_1 = set(list(lieu_aire.index))

In [ ]:
set_2 = set(list(df['lieu']))

In [ ]:
set_2 == set_1

In [ ]:
q7 = """SELECT  df.lieu as lieu,
                df.arrondissement as arrond,
                df.geo_point_2d_a as lat,
                df.geo_point_2d_b as lon,
                x.aire as aire
                FROM df INNER JOIN x ON df.lieu == x.lieu ORDER BY lieu"""

In [ ]:
df_graph = ps.sqldf(q7, locals())

In [ ]:
df_graph

### Création de couleurs de bulles en fonction de chacun des arrondissements

Les bulles de nos lieux seront munies d'une couleur dans notre graphe pour distinguer les arrondissements.

In [ ]:
couleurs_list = ['#B22222',
         '#F08080',
         '#DC143C',
         '#FF0000',
         '#FF4500',
         '#FF8C00',
         '#FFE4B5',
         '#32CD32',
         '#008000',
         '#90EE90',
         '#808000',
         '#7FFFD4',
         '#66CDAA',
         '#48D1CC',
         '#00CED1',
         '#008080',
         '#87CEFA',
         '#1E90FF',
         '#6495ED',
         '#0000FF',
         '#000080',
         '#7B68EE',
         '#EE82EE',
         '#8A2BE2',
         '#FF69B4'
         ]

In [ ]:
couleurs_dict = dict()
for it, val in enumerate(df_graph['arrond'].unique()):
    couleurs_dict.update({val: couleurs_list[it]})

In [ ]:
df_graph['couleurs'] = df_graph['arrond'].map(couleurs_dict)

In [ ]:
df_graph.head(3)

In [ ]:
#data['couleurs'] = data['arrond'].map(couleurs_dict)

Nous exportons notre tableau trier par ordre alphabétique de lieu pour l'utiliser sur notre gpu et pour que nous réalisions le calcul de la matrice des distances entre les lieux.

In [ ]:
df_graph.to_excel('export_df_graph.xlsx')

2. Nous calculerons la matrices des distances entre chaque lieu sur un gpu (le code est fourni en annexe), et ce sera donc une matrice de (6921 x 6921). Une distance est donnée par la relation $\sqrt{((x_1 - x_0)*111)^2 + ((y_1 - y_0)*80)^2} ,\forall x \in latitude, \forall  y\in longitude$ en France, pour la distance en $km$. Le script de calcul de cette matrice est réalisé dans l'IDE PyCharm avec la bibliothèque PyCuda et une carte NVIDIA, localement. L'écriture du code en C ne se prête hélas pas tellement, visuellement parlant, au Notebook de Jupyter.

En sortie nous traitons notre matrice pour la retrouver dans une dataframe avec nos indexes et colonnes en nom de lieu. Comme les calculs ont été réalisés dans des "numpy arrays" les colonnes n'ont pas été nommées. Mais nous avons effectuer les calculs sur un échantillon des données avec notre cpu, puis nous importerons le fichier de résultats de 600Mo dans une DataFrame pour les comparer et voir que nous trouvons le même résultat sur un échantillon.

In [ ]:
distances_cpu = pd.DataFrame(df_graph, index=df_graph['lieu'].unique(), columns=df_graph['lieu'].unique())

In [ ]:
for it, row in enumerate(distances_cpu.iloc[:5,:5].columns):
    for it, col in enumerate(distances_cpu.iloc[:5,:5].columns):
        distances_cpu[col].loc[row] = math.sqrt(((df_graph['lat'].loc[df_graph['lieu'] == col].values - df_graph['lat'].loc[df_graph['lieu'] == row].values)*111)**2\
+ ((df_graph['lon'].loc[df_graph['lieu'] == col].values - df_graph['lon'].loc[df_graph['lieu'] == row].values)*80)**2)

distances_cpu.iloc[:5,:5]

## Code GPU de calcul de la matrice des distances 

Compte-tenu qu'il est nécessaire d'utiliser Anaconda, cela contreviendrait à la consigne qui stipule que l'exercice doit intégrer un environnement virtuel propre. Mais le code est présenté pour preuve :

In [1]:
import pycuda.autoinit
from pycuda import driver, compiler, gpuarray, tools
kernel_code_template = """
__global__ void MatrixMulKernel(double* a, double* b, double* c)
{
 
    for (int k = 0; k <= %(MATRIX_SIZE)s; ++k) {
            for (int l = 0; l <= %(MATRIX_SIZE)s; ++l) {
                    double Aelement = a[l] - a[k];
                    double Belement = b[l] - b[k];
                    c[%(MATRIX_SIZE)s * k + l] = sqrt((Aelement*111)*(Aelement*111) + (Belement*80)*(Belement*80));
                    }
    }


}
"""

matrice = pd.read_excel('export_df_graph.xlsx')
NOMBRE = len(matrice)
MATRIX_SIZE = NOMBRE#len(na.iloc[:NOMBRE])

a_cpu = np.array(matrice['lat'].iloc[:NOMBRE]).reshape((NOMBRE, 1)).astype(np.float64())
b_cpu = np.array(matrice['lon'].iloc[:NOMBRE]).reshape((NOMBRE, 1)).astype(np.float64())

c_cpu = np.dot(a_cpu, b_cpu.T)
a_gpu = gpuarray.to_gpu(a_cpu)
b_gpu = gpuarray.to_gpu(b_cpu)

c_gpu = gpuarray.empty((6921, 6921), np.float64())

kernel_code = kernel_code_template % {
    'MATRIX_SIZE': MATRIX_SIZE
}

mod = compiler.SourceModule(kernel_code)

matrixmul = mod.get_function("MatrixMulKernel")

matrixmul(
    a_gpu, b_gpu,
    c_gpu,
    block=(5, 5, 1),
)

result = c_gpu.get()


col_new = dict()
for x in range(6920):
    col_new.update({x: matrice['lieu'].unique()[x]})

df_result = pd.DataFrame(result)
print(df_result)
df_result = df_result.rename(columns=col_new, index=col_new)
#df_result.to_excel('distances_gpu.xlsx')
df_result = df_result.iloc[:1001,:1001]
df_result.to_excel('distances_gpu.xlsx')

NameError: name 'pd' is not defined

Ici nous importons les résultats des calculs fait à partir de notre gpu local, sur l'heuristique des distances entre lieux. Ce fichier fait 600 Méga Octets.
Nous voyons que les échantillons ci-dessus, et ci-dessous ont des résultats semblables. Comme le chargement du fichier prend trop de temps, nous avons pris le soin de produire un échantillon de 25 lieux, et c'est lui que nous chargeons.

In [ ]:
#distances_gpu = pd.read_excel('distances_gpu.xlsx')
#distances_gpu = pd.read_excel('short_distances.xlsx').drop(['Unnamed: 0'], axis=1).rename(columns={'Unnamed: 0.1':'lieu'})
distances_gpu = pd.read_excel('distances_gpu.xlsx').drop(['Unnamed: 0'], axis=1).rename(columns={'Unnamed: 0.1':'lieu'})

Nous comparons le résultat avec le tableau ci-dessous pour comprendre que tout s'est bien passé sur notre gpu et que nous pouvons travailler avec nos données.

In [ ]:
distances_gpu['lieu'] = distances_gpu.columns

In [ ]:
distances_gpu.index = distances_gpu['lieu']

In [ ]:
distances_gpu.drop(['lieu'], axis = 1, inplace=True)

In [ ]:
distances_gpu.info()

Nous sauvegardons un échantillon de notre matrice pour pouvoir effectuer le programme sur 25 lieux :

In [ ]:
#distances.iloc[:25,:26].to_excel('short_distances.xlsx')

## Problème du voyageur de commerce, algorithme de Christofides

Nous considérons un graphe G(V,E) dont les poids respectent l'égalité triangulaire $d_{ij}+d_{jk}\le d_{ik}$.

La première étape de l'écriture de cet algorithme est l'écriture un arbre couvrant le poids minimum.
Nous vous le présentons ci-dessous :

In [ ]:
SIZE = 50 #  Nombre de lieu à parcourir pour revenir à son point de départ

In [ ]:
d = distances_gpu.iloc[:SIZE,:SIZE].copy()

Voici comment nous retournerons les valeurs des distances entre nos points, ce sont des kilomètres.

In [ ]:
#d.loc[['44 ENFANTS D\'IZIEU'],['28 BOULEVARD DE DOUAUMONT']].values[0][0]# Selectionner à partir de l'index

In [ ]:
#d = distances_gpu.copy()#.iloc[:SIZE,:SIZE+1]

In [ ]:
len([x for i in itertools.permutations([1,2,3])])

In [ ]:
#start = random.choice(d.columns)

In [ ]:
#num_int = 0

In [ ]:
#visite = {}#  villes visitées
#set_visite = set()

In [ ]:
#d.loc[d[start] == d[start].sort_values(ascending=True, ignore_index=True, kind='mergesort').iloc[2]].index[0]# mergesort O(nlog(n))

In [ ]:
#visite.update({num_int: start})# nous ajoutons une ville de départ
#set_visite.add(start)

In [ ]:
#print(visite, set_visite)

In [ ]:
#d[[start]].sort_values(ascending=True,by=[start], axis=0,  kind='mergesort')#.iloc[0].index[1]# mergesort O(nlog(n)))

In [ ]:
#ord_ville = d[[start]].sort_values(ascending=True,by=[start], axis=0,  kind='mergesort')

In [ ]:
#ord_ville.iloc[0:].index[0]

In [ ]:
#set({ord_ville.iloc[0:].index[0]})

In [ ]:
# Nous prenons la ligne de la ville la plus proche
visite = dict()
set_visite = set()
start = random.choice(d.columns)
visite.update({0:start})
set_visite.add(start)
next_ville = start
next_try = ''
km = float()
for pos_chem in range(SIZE):
    next_try = d[[next_ville]].sort_values(ascending=True, by=[next_ville], axis=0,  kind='mergesort')
    for proxima in range(SIZE):#  Pour la taille
        next_ville = next_try.iloc[proxima:].index[0]
        if SIZE == len(visite):
            print('Chemin de poids minimum terminé')
            break
        elif set_visite.isdisjoint(set({next_ville})):# not in visite.values()
            visite.update({pos_chem+1: next_ville})
            set_visite.add(next_ville)
            km += d.loc[[visite[pos_chem]],[visite[pos_chem+1]]].values[0][0]
            break
    if pos_chem == SIZE -1:
        km += d.loc[[visite[pos_chem]],[visite[0]]].values[0][0]
print(visite, 'soit : ', int(km), 'km')

In [ ]:
km

Le résultat du chemin de poids minimal à partir d'un départ aléatoir est présenté ci-dessous

Ensuite nous Calculons l'ensemble des sommets impairs :

In [ ]:
impaire_ensemble = dict()
for v in range(len(visite)):
    if v%2==1:
        impaire_ensemble.update({v: visite[v]}) 

In [ ]:
#impaire_ensemble

Nous calculons un couplage de poids minimum dans l'ensemble des sommets impairs.

In [ ]:
impaire_d = d[list(impaire_ensemble.values())].loc[list(impaire_ensemble.values())]

In [ ]:
#impaire_d

In [ ]:
#impaire_visite = dict()

In [ ]:
#impaire_d[['ALLEE ROYALE']].sort_values(ascending=True, by=[next_ville], axis=0,  kind='mergesort')

In [ ]:
#num_int = 1

In [ ]:
#start = 1

In [ ]:
#column_list = [lieu for v,lieu in impaire_ensemble.items()] 

In [ ]:
#impaire_visite.update({num_int: impaire_ensemble[start]})

In [ ]:
#impaire_dist = impaire_d[impaire_ensemble[start]]

In [ ]:
#len(list(range(len(impaire_d))))

In [ ]:
#impaire_d[[next_ville]].sort_values(ascending=True, by=[next_ville], axis=0,  kind='mergesort')

In [ ]:
# Nous prenons la ligne de la ville la plus proche
imp_visite = dict()
imp_set_visite = set()
start = impaire_d.index[0]
imp_visite.update({1:start})
imp_set_visite.add(start)
next_ville = start
next_try = ''
imp_km = float()
for pos_chem in range(1, SIZE, 2):
    #print(pos_chem)
    next_try = impaire_d[[next_ville]].sort_values(ascending=True, by=[next_ville], axis=0,  kind='mergesort')
    for proxima in range(1,len(impaire_d)):#  Pour la taille
        #print(proxima, len(next_try), next_try.index)
        #print(next_try, proxima)
        next_ville = next_try.iloc[proxima:].index[0]
        if len(imp_visite) == len(impaire_d):
            print('Chemin impair de poids minimum terminé')
            break
        elif imp_set_visite.isdisjoint(set({next_ville})):# not in visite.values()
            imp_visite.update({pos_chem+2: next_ville})
            imp_set_visite.add(next_ville)
          #  print(pos_chem, imp_visite[pos_chem])
            imp_km += impaire_d.loc[[imp_visite[pos_chem]],[imp_visite[pos_chem+2]]].values[0][0]
            break
print(imp_visite, 'soit : ', int(imp_km), 'km')

Nous devons faire l'union du couplage ci-dessus et de l'arbre couvrant de poids minimum.

In [ ]:
#union

final = {}
union = deque()
for k,v in visite.items():
    if k%2==0:
        union.append({k: v})
    if k%2==1:
        if v == imp_visite[k]:
            union.append({k: v})
        if v != imp_visite[k]:
            double_edge = set({v, imp_visite[k]})
            union.append({k: double_edge})
union

In [ ]:
final = {}
union = deque()
for k,v in visite.items():
    if k%2==0:
        union.append(v)
    if k%2==1:
        if v == imp_visite[k]:
            union.append( v)
        if v != imp_visite[k]:
            double_edge = list()
            double_edge.append(v)
            double_edge.append(imp_visite[k])
            union.append(double_edge)
union = pd.DataFrame(list(union), columns=['lieu'])

In [ ]:
a = union.iloc[3]#.loc[union['lieu'] == '{ANDRE BRECHET (21) MAT, ALBIN HALLER 5}']

In [ ]:
a[0]

In [ ]:
euler_tour = [ a[0][0], a[0][1], union.iloc[4][0]]

In [ ]:
[p for p in itertools.permutations(list(euler_tour))]


In [ ]:
final = {}
union = deque()
for k,v in visite.items():
    if k%2==0:
        union.append({k: v})
    if k%2==1:
        if v == imp_visite[k]:
            union.append({k: v})
        if v != imp_visite[k]:
            double_edge = set({v, imp_visite[k]})
            union.append({k: double_edge})
#n = 0 # to continue one loop to increment the deque
km_final = float(0)
km = float(0)
km_second = float(0)
i = 0
for n in range(SIZE):
    print('i ',i)
    chemin = union[i]
    # Pour les deux premiers, pas d'impair, ce sont des strings
    if i<=1:
        final.update({i: chemin[i]})
        i+=1
    elif isinstance(chemin[i], str):
        voie_0 = list(chemin.values())[0]
        #final.update({i: chemin[i]})
        if i>1:
            if voie_0 in list(final.values()):
                print('merde')
                i+=1
                continue
            km_final += d.loc[[final[i-1]],[voie_0]].values[0][0]
            final.update({i-1: final[i-1], i: voie_0})
            print(i,final[i-1], voie_0)
            print(i,final[i-1],final[i])
        # Comme le premier est distant de zéro avec lui-même
        # nous prenons ici le cas de la mesure de la première distance
        elif i==1:
            km_final += d.loc[[final[i-1]],[final[i]]].values[0][0]
        i+=1
    # Si nous avons un impair, nous affectons les 4 valeurs en entrées
    elif isinstance(chemin[i], set):
        #print(final[i-1])
        print('i : ',i, chemin[i])
        voie_0 = final[i-1] # Cette valeur est le dernier lieu visité
        voie_1 = chemin[i].pop()
        voie_2 = chemin[i].pop()
        if voie_1 in list(final.values()):
            if voie_2 in list(final.values()):
                i+=1
                continue
            else:
                final.update({i: voie_2})
                km_final += d.loc[[final[i-1]],[final[i]]].values[0][0]
                print('none : ', n)
                i+=1
                continue
        elif voie_2 in list(final.values()):
            if voie_1 in list(final.values()):
                print('none : ', n)
                i+=1
                continue
            else:
                final.update({i: voie_1})
                km_final += d.loc[[final[i-1]],[final[i]]].values[0][0]
                i+=1
                continue
            #final.update({i: voie_2})
            #km_final += d.loc[[final[i-1]],[final[i]]].values[0][0]
            #continue
            
        # Lorsque le tour arrive à la fin, il revient au début
        if i < SIZE - 1:
            voie_3 = list(union[i+1].values())[0]
        if i == SIZE - 1:
            voie_3 = final[0] # le dernier lieu et le premier sont les mêmes
        
        sort_tri = set()
        euler_tour = [voie_1, voie_2, voie_3]
        euler_tour = {p for p in itertools.permutations(euler_tour)}
        sort_tri.add(voie_1)
        sort_tri.add(voie_2)
        sort_tri.add(voie_3)
        km = d.loc[[voie_0],[voie_1]].values[0][0] + d.loc[[voie_1],[voie_2]].values[0][0] + d.loc[[voie_2],[voie_3]].values[0][0]
        
        for k in range(len(sort_tri)):
            final.update({i+k: sort_tri.pop()})
            
        for j in range(5):
            tour_ = euler_tour.pop()
            voie_1_second = tour_[0]
            voie_2_second = tour_[1]
            voie_3_second = tour_[2]
            km_second = d.loc[[voie_0],[voie_1_second]].values[0][0]+ d.loc[[voie_1_second],[voie_2_second]].values[0][0]\
                + d.loc[[voie_2_second],[voie_3_second]].values[0][0]
            
            if km > km_second:
                sort_tri = set()
                sort_tri.add(voie_1_second)
                sort_tri.add(voie_2_second)
                sort_tri.add(voie_3_second)
        for k in range(len(sort_tri)):
            final.update({i+k: sort_tri.pop()})
        final.update({i-1: voie_0})
        print(i, voie_0, voie_1, voie_2, voie_3)
        print(i,final[i-1],final[i],final[i+1],final[i+2])
        km_final += d.loc[[final[i-1]],[final[i]]].values[0][0]+ d.loc[[final[i]],[final[i+1]]].values[0][0]\
                + d.loc[[final[i+1]],[final[i+2]]].values[0][0]
        i+=1
print(final, km_final)

In [ ]:
x = pd.Series(final.values())

In [ ]:
x.loc[x.duplicated()]

chemin[1].pop()

chemin = 'ALLEE DES JUSTES DE FRANCE'
voie_1 = '48 BOULEVARD DE DOUAUMONT'
voie_2 = 'ALLEE VIVALDI'
voie_3 = 'ALLEE DES LAPINS'
euler_tour = [voie_1, voie_2, voie_3]
euler_tour = {p for p in itertools.permutations(euler_tour)}

len (euler_tour)

tour_ = euler_tour.pop()

tour_

d.loc[[voie_1],[voie_2]].values[0][0]

sort_tri = set()
km = float
chemin = 'ALLEE DES JUSTES DE FRANCE'
voie_1 = tour_.pop()
voie_2 = tour_.pop()
voie_3 = tour_.pop()
sort_tri.add(voie_1)
sort_tri.add(voie_2)
sort_tri.add(voie_3)
km = d.loc[[chemin],[voie_1]].values[0][0]+ d.loc[[voie_1],[voie_2]].values[0][0] + d.loc[[voie_2],[voie_3]].values[0][0]
for i in range(5):
    voie_1_second = tour_.pop()
    voie_2_second = tour_.pop()
    voie_3_second = tour_.pop()
    km_second = d.loc[[chemin],[voie_1_second]].values[0][0]+ d.loc[[voie_1_second],[voie_2_second]].values[0][0]\
                + d.loc[[voie_2_second],[voie_3_second]].values[0][0]
    print(sort_tri, km, km_second)
    if km > km_second:
        sort_tri = set()
        sort_tri.add(voie_1_second)
        sort_tri.add(voie_2_second)
        sort_tri.add(voie_3_second)
print(sort_tri, km, km_second)

Calculons le tour Eulérien et le tour le plus court en même temps :

print(lieu_un, lieu_deux)

len(union)

chemin = OrderedDict()
for i in range(4):
    enter = union.popitem(last=False)
    if isinstance(enter[1], str):
        lieu_1 = enter
    elif isinstance(enter[1], set):
        lieu_2 = enter[1].pop()
        lieu_3 = enter[1].pop()
        out = union.popitem(last=False)
        
        print([p for p in itertools.permutations([lieu_1,lieu_2,lieu_3,lieu_4])])


SIZE = 25 #  Nombre de lieu à parcourir pour revenir à son point de départ

d = distances.iloc[:SIZE,:SIZE+1]

start = union[0]

num_int = 0

christofides_solution = {}#  ville visitée

christofides_solution.update({num_int: start})

christ_dist = d[start].sort_values(ascending=True, ignore_index=True, kind='mergesort')

isinstance(union[3], set)

euler_tour = set()

euler_tour.add(union[4])

euler_tour.add(union[3].pop())

{p for p in itertools.permutations(euler_tour)}

d[union[1]].loc[d['lieu'] == union[3].pop()]

for i in range(SIZE-1):
    n = 0
    while True:#  si lieu déjà visité
        if SIZE == len(christofides_solution):
            print('Voyage terminé')
            break
        elif isinstance(union[n], set):
            euler_tour = set()
            euler_tour.add(union[n+2])
            for i in range(2):
                euler_tour.add(union[n+1].pop())
            {p for p in itertools.permutations(euler_tour)}

for i in range(SIZE-1):
    n = 0
    while True:#  si lieu déjà visité
        if SIZE == len(christofides_solution):
            print('Voyage terminé')
            break
        # Si c'est la première fois que l'on visite ce lieu    
        elif d['lieu'].loc[d[christofides_solution[num_int]] == christ_dist[n]].ravel()[0] not in christofides_solution.values():
            # Rajouter le nom du lieu le plus proche dans la liste des solutions
            christofides_solution.update({num_int + 1 : d['lieu'].loc[d[christofides_solution[num_int]] == christ_dist[n]].ravel()[0]}) 
            break
        else :
            n = n + 1
    num_int = num_int + 1
    christ_dist = d[christofides_solution[num_int]].sort_values(ascending=True, ignore_index=True, kind='mergesort')

Ainsi, nous avons présenté les données de la ville de Paris, nous avons effectué une analyse univariée sur les éléments chiffrés, à la suite de quoi, nous avons calculé la matrice des distances entre les lieux, en ayant éliminé les doublons. Pour la présentation future du graphe, nous avons rajouté le nombre d'arbres par lieu dans la colonne aire, et une couleur distincte par arrondissement. Enfin nous avons résolu algorithmiquement le chemin de poids minimum pour n'importe quel point de départ. Par contre l'algorithme de Christofides étant de complexité $O(x) = x^3$, il me semble bien ambitieux pour 6921 lieux, et donc aussi pour le calculer dans ce projet dans des délais raisonnables.

## Optimisation des ressources

In [ ]:
cvs = ds.Canvas(plot_width=850, plot_height=500)
agg = cvs.points(df_graph, 'lon', 'lat')
img = ds.tf.shade(agg, cmap=list(df_graph['couleurs']), how='eq_hist')
img